# Convert a spreadsheet to a website.

A couple of rules:

Name your pages like ```App layout``` or ```Shop layout```, with an optional ```Shop data``` which will automatically load as a data table to be used in the template like ```{{#shop-data}}{{description}}{{/shop-data}}```. This would repeat for every row in the data table printing out all of the description columns. Data tables should list the column names as the first row. Hiding sheets doesn't matter to the converter but makes it easier to manage lots of pages or see only the one's you are working on.

Use HTML or Mustache or Markdown in your sheets. Rows are automatically converted to top level ```<div>``` elements. The class ```col-2``` is added to count the number of columns. Cells are automatically converted to ```<div>``` as children on the row ```<div>```. The class ```cell-0``` is automatically added as the index of the cell, not including any variables in between.

Beginning a cell with ":" comma creates variables and the cell right of a variable is used at the value. Variables do not affect column count or HTML output. Variables are combined with subpages, that is, any variable set on one page can be used page another page loaded by a URL, ```{{> section}}```, or ```::render```.

Special variables are:

```:logo```, which shows up in the favicon.

```:title``` is used as page title.

```::render``` forces another template to render whether or not there is a link to it, this is a list and can be used as many times as you want.

```{{> section}}``` inserts a {{#section}}{{/section}} mustache template, combines section from mustach and render from this framework. Pages that are not linked to, rendered, or sectioned, won't be scanned at all. Sections should be refered to in lowercase with spaces converted to dashes and without the word ```layout``` attached, i.e. ```{{> shop-menu}}```.

**Now for the complicated part!** ```/groups/1``` will load the layout ```groups``` and filter by the ```url``` or ```link``` column to match a single group. If no ```link``` or ```url``` column matches, it will search the column named after the page, the ```groups``` column will be used to match data categorically, all data rows matching ```/group/1``` will be available for repetion with a ```{{#groups-data}}{{/groups-data}}``` section tag.

The same rule applies to section includes, but even better! You can include a filtered section from the parents match, if you had groups and subgroups for each of the groups. ```{{> subgroup/groups}}``` will use the ```{{groups}}``` variable from the current page, and filter subgroups accordingly. Any subgroup with the subgroup matching ```/group/1``` will be available for repetition using ```{{#subgroup/groups-data}}{{/subgroup/groups-data}}```.

URLs such as ```/link``` inherently forces the spreadsheet page named Link layout, or Link data to render. Links to other pages are automatically detected. Names of pages are automatically converted to lowercase, with dashes instead of spaces.

Use ```::variable``` to create a list such as ```::stylesheet``` or ```:scripts```.

[Markdown description at markdownguide.org](https://www.markdownguide.org/basic-syntax)

[Mustache guide from NodeJs](https://github.com/janl/mustache.js)


TODO:

Inlining all styles and scripts and images as data-uri for faster load times.

Automatically detecting which features to add such as ```Buy Now``` buttons and contact us chats.

readme.md?


In [ ]:
// placeholder for readme

filter data sheet based on url?

In [ ]:
var TRIM = /^\s*\/\s*|\s*\/\s*$/ig;

var compareLink = (dataValue, base, link) => {
    return (dataValue || '').replace(TRIM, '') === link
        || (dataValue || '').replace(TRIM, '') === base + '/' + link
        || dataValue === link.split('/').slice(1).join('/')
        || dataValue === base
}

function unfilteredData(key) {
    return (val, render) => render(`{{#${key}-original-data}}${val}{{/${key}-original-data}}`)
}

function filteredData(key, match, properties, categorical) {
    return function(val, render) {
        var link = (render(`{{{${match}}}}`) || properties[match] || '').replace(TRIM, '');
        var base = (render(`{{{base}}}`) || properties['base'] || '').replace(TRIM, '');
        if(link.substr(0, base.length) === base) {
            link = link.substr(base.length).replace(TRIM, '');
        }
        var matchKey = key + '-filtered-data';
        var restore = properties[key + '-original-data'];
        
        if(categorical) {
            // handle multiple results for use with categories
            properties[matchKey] = restore.filter(data => compareLink(data[key], base, link))
        } else {
            properties[matchKey] = [restore.filter(data => compareLink(data['link'], base, link)
                                                   || compareLink(data['url'], base, link))[0]];
            if(typeof properties[matchKey][0] === 'undefined') {
                throw new Error(`Unique key not found: ${key} ${match} ${link}`)
            }
        }
        
        console.log(`Rendering filtered ${categorical 
                    ? 'categorical' 
                    : 'unique'}: ${key} ${match} ${link} ${properties[matchKey].length}`);
        var rendered = render(`{{#${matchKey}}}${val}{{/${matchKey}}}`);
        delete properties[matchKey];
        return rendered;
    }
}

module.exports = {
    filteredData,
    unfilteredData
}


google sheet template properties?


In [ ]:
var importer = require('../Core');
var getDataSheet = importer.import('google sheet array objects');
var renderRows = importer.import('google sheet layout template');
var getRows = importer.import('get worksheet rows');
var {filteredData, unfilteredData} = importer.import('filter data sheet based on url');

var isCategorical = (data, key) => data.filter(row => row.hasOwnProperty(key)).length > 0;

var isWrapper = rows => rows.length === 1 && rows[0].length === 1
              && rows[0][0].match(/\{\{> (.*?)\/(.*?)-(.*?)-link\}\}/ig);

// TODO: make a style notebook for patterns like this
// similar to lazy loading
function promiseOrResolve(obj, property, cb) {
    return obj ? (typeof obj[property] != 'undefined'
                  ? Promise.resolve(obj[property])
                  : cb(obj).then(d => (obj[property] = d)))
        : Promise.resolve()
}

function getTemplateProperties(key, properties, templates) {
    if(typeof templates[key] === 'undefined') {
        throw new Error(`section ${key} not found!`)
    }
    // load template data
    var rows;
    return promiseOrResolve(templates[key].data, 'rows', getDataSheet)
        .then(data => {
            properties[key + '-original-data'] = data;
            properties[key + '-data'] = unfilteredData.bind(null, key);
        })
        // load template layout
        .then(() => promiseOrResolve(templates[key].template, 'rows', getRows))
        .then(rs => (rows = rs || []).flat()
              .reduce((p, c, j) => p.then(() => matchSections(c, properties, templates)), Promise.resolve()))
        // detect if this is just a wrapper template and don't render rows, do a direct replacement
        .then(() => isWrapper(rows) ? rows[0][0] : renderRows(key, rows, properties, templates))
        .then(template => (properties[key] = template))
}

// must do this up front so we can process all data
var matchSections = (cell, properties, templates) => importer
    .regexToArray(/\{\{>\s*(.*?)\}\}/ig, cell, 1)
    .reduce((promise, section) => promise
        .then(() => getTemplateProperties(section.split('/')[0], properties, templates))
        .then(() => section.split('/').length > 1
              ? createAssignFilter(section, properties)
              : Promise.resolve()),
            Promise.resolve())

function createAssignFilter(section, properties) {
    var key = section.split('/')[0];
    var match = section.split('/').slice(1).join('/');
    properties[section] = properties[key];
    
    // add a special partial for the filtered data
    var categorical = isCategorical(properties[key + '-original-data'] || [])
    // automatically wrap unique templates in a data section for accessing filtered properties
    if(!categorical && !properties[key].includes(`{{#${key}-data}}`)
        && typeof properties[key + '-original-data'] !== 'undefined') {
        properties[section] = `{{#${key}-data}}${properties[key]}{{/${key}-data}}`;
    }
    // run the filtered function instead of using the array
    properties[key + '-data'] = filteredData.bind(null, key, match, properties, categorical);
}

module.exports = getTemplateProperties;


google sheet layout template?

Convert a google sheet page to an HTML template



In [ ]:

function safeName(name) {
    return name.replace(/[^a-z0-9\-]/ig, '-').substr(0, 40);
}

function escape(s) {
    return s.replace(/[-\/\\^$*+?.()|[\]{}]/g, '\\$&');
}

function getDataClasses(c, data) {
    // get classes from mustache vars used with supplied data
    if(typeof data != 'object') {
        return [];
    }
    var dataKeys = (data || []).reduce((keys, cur) => {
        // get all keys in data
        return keys.concat(Object.keys(cur));
    }, []).filter((k, h, a) => a.indexOf(k) == h);
    
    return dataKeys
        .filter(k => c.match(new RegExp('\\{\\{\\s*[>#\\/]?\\s*' + escape(k) + '\\s*\\}\\}', 'ig')))
}

function defineProperty(c, value, properties) {
    if (c.substr(0, 2) === '::' || c === ':render') {
        if(c === ':render') c = '::render'; // just to fix using it below
        if(typeof properties[c.substr(2)] == 'undefined') {
            properties[c.substr(2)] = [];
        }
        properties[c.substr(2)][properties[c.substr(2)].length] = value;
    } else {
        properties[c.substr(1)] = value;
    }
}

function renderRows(layoutTitle, rows, properties, templates) {
    if(!rows) {
        rows = [];
    }
    // set object properties for mustache template
    var html = rows.reduce((arr, row, i) => {
        var rowsHtml = row.reduce((arr, c, j) => {
            var dataClasses = getDataClasses(c, properties[layoutTitle + '-original-data'])
                .map(k => 'val-' + safeName(k))
                .join(' ')
            var sectionClasses = getDataClasses(c, [properties, templates])
                .map(k => 'section-' + safeName(k))
                .join(' ')
            if(c.substr(0, 1) === ':') {
                // use subsequent column for property values
                defineProperty(c, row[j + 1], properties)
                
            // render if it is not the value for the previous property
            } else if(j == 0 || j >= 1 && row[j - 1] && row[j - 1].substr(0, 1) !== ':') {
                arr[arr.length] = `
<div class="cell-${arr.length} ${dataClasses} ${sectionClasses}">
${c}
</div>
`;
            }
            return arr;
        }, []);

        // render mustache templates
        if(rowsHtml.length > 0) {
            arr[arr.length] = `
<div class="row-${arr.length} ${properties['class'] || ''} col-${rowsHtml.length}">
${rowsHtml.join('')}
</div>
`;
        }

        return arr;
    }, []);

    // render mustache page template
    return html.join('');
}

module.exports = renderRows;



output google sheet template?

Save the generated template to an HTML file, wrapping it in a base template


In [ ]:
var fs = require('fs');
var path = require('path');
var Mustache = require('mustache');
var Remarkable = require('remarkable');

function safeName(val, render) {
    return render(val).replace(/[^a-z0-9\-]/ig, '-').substr(0, 40)
}

function toJSON(val, render) {
    return render(JSON.stringify(val))
}

function wrapTemplate(path, html, properties) {
    properties['safeName'] = () => safeName;
    properties['toJSON'] = () => toJSON;

    var domain = '';
    if(typeof properties['domain'] != 'undefined') {
        domain = properties['domain'].includes(':') ? '{{domain}}' : 'https://{{domain}}';
    }
    
    // automatically set title if it isn't set manually
    var result;
    if(typeof properties['title'] == 'undefined' && (result = (/<h1>(.*)<\/h1>/ig).exec(html))) {
        properties['title'] = result[1];
    }
    
    var pageHtml = `
<!doctype html>
<html>
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=no">
<link rel="icon" href="{{logo}}">
{{#stylesheet}}<link rel="stylesheet" media="screen" href="{{.}}">{{/stylesheet}}
{{#base}}<base href="/{{.}}" />{{/base}}
<meta property="og:type" content="website">
<meta property="og:title" content="{{title}}">
<link rel="canonical" href="${domain}/${path}">
<title>{{title}}</title>
<style>
body > div.col-1:nth-of-type(2) {
{{#banner}}background-image: url({{{.}}});{{/banner}}
}
</style>
</head>
<body class="${path.replace(/\//ig, ' ')} {{class}}">
${html}
{{#script}}<script src="{{.}}"></script>{{/script}}
</body>
</html>`;
    Mustache.parse(pageHtml);
    // use properties for view and for partials
    return Mustache.render(pageHtml, properties, properties);
}

module.exports = wrapTemplate;



find known routes to sheets?


In [ ]:
var importer = require('../Core');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var {getTemplateByUrl, getEntryTemplate} = importer.import('convert sheet helper functions');

// combine with "getSections" by using fake "{{> url/path}}" include
function collectRoutes(routes, properties, templates, rendered) {
    var local = routes.concat(properties['render'] || [])
        .filter((link, i, arr) => !rendered.includes(link)
                // protocol means it's absolute remote path and not to try to generate it
                && arr.indexOf(link) == i && !link.includes(':'))

    local.forEach(link => rendered[rendered.length] = link)
    
    return importer.runAllPromises(local
        // promise in series so there is no data collisions
        .map(link => resolve => {
            link = link.replace(/^\/|\/$/ig, '');
            var trimmedBase = (properties['base'] || '').replace(/^\/|\/$/ig, '');            
            if(link.substr(0, trimmedBase.length) === trimmedBase) {
                link = link.substr(trimmedBase.length).replace(/^\/|\/$/ig, '');
            }
            // any part of a path can contain the reference to a page template
            var key = getTemplateByUrl(templates, link);
            var newProps = Object.assign({}, properties);
        
            // create a temporary template to filter by
            newProps[key + '-' + key + '-link'] = link;
            templates[key + '-' + key] = {template: {rows: [[`{{> ${key}/${key}-${key}-link}}`]]}}
        
            return getTemplateProperties(key + '-' + key, newProps, templates)
                .then(() => wrapTemplate(link, newProps[key + '-' + key], newProps))
                .then(page => {
                    var pages = {};
                    pages[link] = page;
                    resolve(pages)
                })
        }))
        .then(results => results.reduce((obj, r) => Object.assign(obj, r), {}))
}

module.exports = collectRoutes;



collect external content and resources?


In [ ]:
var {Readable} = require('stream');
var {JSDOM, XPathResult} = require('jsdom');
var importer = require('../Core');
var getArrayXPath = importer.import('get xpath array');

function safeName(name) {
    return name.replace(/[^a-z0-9\-]/ig, '-').substr(0, 40);
}

function collectExternalResources(page, rendered, routes) {
    
    // get all images and urls from template
    var dom = new JSDOM(page);
    
    // add IDs to h1, h2, h3, etc elements that match their text contents
    var headingsObjs = getArrayXPath('(//h1|//h2|//h3|//h4)[not(@id)]', dom.window.document.body);
    headingsObjs.forEach(h => h.setAttribute('id', safeName(h.textContent)));
    
    var linksObjs = getArrayXPath('//a[@href]', dom.window.document.body);
    var links = linksObjs.map(l => l.getAttribute('href'));
    
    // TODO: convert images and add timestamps, add svg
    var imgObjs = getArrayXPath('//img[@src]', dom.window.document.body);
    var imgs = imgObjs.map(l => l.getAttribute('src'));
    
    // TODO: scan for urls and inline
    var stylesObjs = getArrayXPath('//link[@href]', dom.window.document.body);
    var styles = stylesObjs.map(l => l.getAttribute('href'));

    // TODO: add timestamps and inline
    var scriptsObjs = getArrayXPath('//script[@src]', dom.window.document.body);
    var scripts = scriptsObjs.map(l => l.getAttribute('src'));
    
    // TODO: add CSS imports
    var backgrounds = importer.regexToArray(/url\((.*?)\)/ig, page, 1);
    
    var searches = imgs.concat(styles).concat(backgrounds).concat(scripts)
        .filter((cur, i, arr) => arr.indexOf(cur) == i && rendered.indexOf(cur) === -1);
    
    links.forEach(s => routes[routes.length] = s)
    searches.forEach(s => rendered[rendered.length] = s)
    
    // TODO: copy resource images to output directory
    var newPage = dom.window.document.documentElement.outerHTML
    var stream = new Readable();
    stream.push(newPage);
    stream.push(null);
    return Promise.resolve(stream);
}

module.exports = collectExternalResources;


convert sheet helper functions?


In [ ]:

var getTemplateByUrl = (templates, path) => !path || path === '' || path === '/'
    ? getEntryTemplate(templates)
    : path.split('/').filter(segment => templates[segment]
                             && templates[segment].template)[0];

var getEntryTemplate = (templates) => Object.keys(templates).filter(t => templates[t].template
                                                     && templates[t].template.properties
                                                     && templates[t].template.properties.index == 0)[0];

module.exports = {
    getTemplateByUrl,
    getEntryTemplate
}

collect google sheets resources?


In [ ]:
var importer = require('../Core');
if((process.env.ENVIRONMENT || '').includes('LOCAL')
   || (process.env.ENVIRONMENT || '').includes('TEST')) {
    var streamToGoogle = importer.import('test stream to output');
} else {
    var streamToGoogle = importer.import('upload files google cloud');
}
var copyFileBucket = importer.import('copy file bucket storage');
var collectExternalResources = importer.import('collect external content and resources');
var collectRoutes = importer.import('find known routes to sheets');
var {getTemplateByUrl, getEntryTemplate} = importer.import('convert sheet helper functions');

var timestamp = (new Date()).getTime();

// detect links and write out every part of the site
function collectTemplateResources(path, page, properties, templates, bucketName, rendered) {
    if(!rendered) {
        rendered = [];
    }
    
    // if it is the first page in the template, rename it to index.html
    if(getTemplateByUrl(templates, path) === getEntryTemplate(templates)) {
        console.log(`using ${path} as index.html`);
        path = 'index';
    }
    
    var trimmedBase = (properties['base'] || '').replace(/^\/|\/$/ig, '');
    if(path.substr(0, trimmedBase.length) !== trimmedBase) {
        path = trimmedBase + '/' + path;
    }

    // TODO: add timestamps to generated content
    // TODO: set different permissions on files streamed to google.
    
    var routes = [];
    return collectExternalResources(page, rendered, routes)
        .then(stream => (console.log(`emitting ${path}`), stream))
        .then(stream => streamToGoogle(path + '.html', bucketName, stream, {
            contentType: 'text/html; charset=utf-8'
        } /* TODO: insert permission settings for user directory */))
        .then(() => copyFileBucket(bucketName, path + '.html'))
        .then(() => collectRoutes(routes, properties, templates, rendered))
        .then(pages => importer.runAllPromises(Object.keys(pages).map(fileName => resolve => 
            collectTemplateResources(fileName, pages[fileName], properties, templates, bucketName, rendered)
                .then(() => resolve()))))
        .then(() => rendered)
}

module.exports = collectTemplateResources;


test google sheets resources?


In [1]:
var assert = require('assert');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var getEnvironment = importer.import('get environment');
getEnvironment('STUDY_LOCAL');
var importSheet = importer.import('sheet marketing import');
var findSimilarFile = importer.import('find similar file');
if((process.env.ENVIRONMENT || '').includes('LOCAL')
   || (process.env.ENVIRONMENT || '').includes('TEST')) {
    var streamToGoogle = importer.import('test stream to output');
} else {
    var streamToGoogle = importer.import('upload files google cloud');
}

var DOWNLOAD_DIR = path.resolve(path.join(__dirname, '../../Downloads'));

var findAllFiles = resources => Promise.all(resources
    .filter(r => !r.includes(':') && r.includes('.'))
    .map(r => findSimilarFile(path.basename(r), DOWNLOAD_DIR)))

var copyAllFiles = (resources, bucketName) => Promise.all(resources
        .map(f => streamToGoogle(f,
                                 bucketName,
                                 fs.createReadStream(path.join(DOWNLOAD_DIR, f)))))

function importTest(link, domain) {
    return importSheet(link, domain)
        .then(resources => findAllFiles(resources))
        .then(resources => copyAllFiles(resources, domain))
}

module.exports = importTest;



SyntaxError: missing ) after argument list

google sheet handler?

get sheet identifier from link?


In [ ]:
var util = require('util');
var uuid = require('uuid/v1');
var getInfo = importer.import('get google sheet info');
var addRow = importer.import('add row data google sheet');
var updateRow = importer.import('update a row in google sheets')

var project = 'spahaha-ea443';

function safeName(name) {
    return name.replace(/[^a-z0-9\-]/ig, '-').substr(0, 40).toLowerCase();
}

function addSheet(docId, title, email) {
    var name = safeName(title.replace(/\s*(configuration|config)\s*/ig, ''))
        + '-' + uuid().substr(0, 5);
    return addRow(purchaseId, {
        timestamp: Date.now(),
        name: title,
        email: email || '',
        address: '',
        domain: '',
        bucket: name + '.sheet-to-web.com',
        project: project,
        sheet: docId
    }).then(() => name + '.sheet-to-web.com')
}

function getSheet(link, email) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var title;
    
    return getInfo(link)
        // return assigned subdomain
        .then(info => title = info.properties.title, getPurchases(docId))
        .then(match => !match
              ? addSheet(docId, title, email)
              : updateRow(purchaseId, r => r.sheet == docId, {name: title, email}))
        .then(row => row.domain || row.bucket)
}

module.exports = getSheet;



get sheet purchases?

In [ ]:
var importer = require('../Core');
var getDataSheet = importer.import('google sheet array objects');

var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';

var isInvalidDomain = (match, domain) =>
    !match || domain !== match.domain && domain != '' && domain !== match.bucket

function getPurchases(docId, domain) {
    return getDataSheet(purchaseId, 'Purchases')
        .then(purchases => purchases.filter(p => p.sheet == docId)[0])
        .then(match => {
            if(domain && isInvalidDomain(match, domain)) {
                throw new Error(`sheet ${docId} doesn't match domain ${domain}`)
            }
            console.log(`purchase ${docId} already exists: ${match.domain} or ${match.bucket}`);
            return match;
        })
}

module.exports = getPurchases;

sheet marketing import handler?

sheet marketing import?


In [ ]:
var importer = require('../Core');
var getPurchases = importer.import('get sheet purchases');
var getTemplates = importer.import('templates google sheet');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var collectTemplateResources = importer.import('collect google sheet resources');

var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';

function importSheet(link, domain) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var properties = {}, templates, key;
    
    return getPurchases(docId, domain)
        .then(match => getTemplates(docId))
        .then(t => {
            templates = t;
            key = Object.keys(templates).filter(k => templates[k].template)[0];
            return getTemplateProperties(key, properties, templates)
        })
        .then(() => wrapTemplate(key, properties[key], properties))
        .then(page => collectTemplateResources(key, page, properties, templates, domain || match.bucket))
        .then(resources => {
            console.log(resources);
            return resources;
        })
}

module.exports = importSheet;



sheet backend handler?

setup sheet backend?


In [ ]:
var importer = require('../Core');
var getPurchases = importer.import('get sheet purchases');
var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';
var addIP = importer.import('check dns');
var {
    insertBackendBucket,
    insertGlobalForward,
    updateUrlMap
} = importer.import('add google bucket web map');

var urlMap = 'web-map';

function setupBackend(link, domain) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var project;

    return getPurchases(docId, domain)
        .then(match => {
            project = match.project;
            domain = domain || match.bucket;
            return addIP(project, domain);
        })
        .then(ip => insertGlobalForward(project, ip, urlMap, domain))
        .then(() => insertBackendBucket(project, domain))
        .then(() => updateUrlMap(project, urlMap, domain))
        .then(() => domain)
}

module.exports = setupBackend;



create a sheet handler?

create a copy of marketing template?


In [ ]:
var uuid = require('uuid/v1');
var importer = require('../Core');
var getSheet = importer.import('get sheet identifier');
var copyFile = importer.import('copy a file on google drive');
var listDrive = importer.import('list google drive files');
var insertPermission = importer.import('insert google drive permissions');

function copyMarketing(email) {
    var fileId;
    return listDrive()
        .then(r => fileId = r.filter(f => f.name === 'Marketing site')[0].id)
        .then(() => copyFile(fileId, 'Marketing site ' + uuid().substr(0, 5)))
        .then(id => insertPermission(id, email))
        .then(() => getSheet(fileId, email))
        .then(() => fileId)
}

module.exports = copyMarketing;



package.json?


In [ ]:
{
    "name": "SheetToWeb",
    "description": "Marketing site functions",
    "license": "UNLICENSED",
    "scripts": {
    },
    "engines": {
        "node": ">= 8",
        "npm": ">= 4"
    },
    "repository": {
        "type": "git",
        "url": "git+https://github.com/megamindbrian/jupytangular.git"
    },
    "dependencies": {
        "@google-cloud/compute": "^0.12.0",
        "@google-cloud/storage": "^2.5.0",
        "googleapis": "^39.2.0",
        "jsdom": "^14.0.0",
        "mustache": "^3.0.1",
        "remarkable": "^1.7.1"
    }
}
